<a href="https://colab.research.google.com/github/yoshinodev/Movies_Data_Analysis/blob/main/finalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Final**

## FASE 1 ***

## Google drive

In [739]:
# Access google drive to upload dataset
from google.colab import drive

drive.mount('/content/drive')

#https://saturncloud.io/blog/how-to-read-csv-to-dataframe-in-google-colab/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Libs

In [740]:
 # Import the necessary libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import ast

## Upload and read dataset

In [741]:
# Upload and read dataset, now stored in variable movies
movies = pd.read_csv('/content/drive/MyDrive/10811_Final_Project/dataset/movies_metadata.csv', usecols=["id", "belongs_to_collection", "budget", "genres", "original_title", "production_companies", "release_date", "revenue", "runtime", "title"], low_memory=False)

Nota: atenção confirmar na coluna genres se eliminou também os "[]" e se existe outra célula inválida. Nota2: genres terá que ser filtrado para conter apenas os nomes separado por vírgulas.

## Summary of dataset in analysis

In [742]:
# movies.info() will show a concise summary of the dataset
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_title         45466 non-null  object 
 5   production_companies   45463 non-null  object 
 6   release_date           45379 non-null  object 
 7   revenue                45460 non-null  float64
 8   runtime                45203 non-null  float64
 9   title                  45460 non-null  object 
dtypes: float64(2), object(8)
memory usage: 3.5+ MB


## Dataset headings and first 5 rows

In [743]:
# method head used to quickly check the data's structure
movies.head

<bound method NDFrame.head of                                    belongs_to_collection    budget  \
0      {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1                                                    NaN  65000000   
2      {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3                                                    NaN  16000000   
4      {'id': 96871, 'name': 'Father of the Bride Col...         0   
...                                                  ...       ...   
45461                                                NaN         0   
45462                                                NaN         0   
45463                                                NaN         0   
45464                                                NaN         0   
45465                                                NaN         0   

                                                  genres      id  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...     

## Check how many null values are

In [744]:
movies.isnull().sum()

belongs_to_collection    40972
budget                       0
genres                       0
id                           0
original_title               0
production_companies         3
release_date                87
revenue                      6
runtime                    263
title                        6
dtype: int64

## Remove null values

In [745]:
# Remove null values from the dataset with the method dropna()
movies = movies.dropna()

## Check if the nulls were correctly removed from the dataset

In [746]:
movies.isnull().sum()

belongs_to_collection    0
budget                   0
genres                   0
id                       0
original_title           0
production_companies     0
release_date             0
revenue                  0
runtime                  0
title                    0
dtype: int64

## Check the data types from the dataset

In [747]:
movies.dtypes

belongs_to_collection     object
budget                    object
genres                    object
id                        object
original_title            object
production_companies      object
release_date              object
revenue                  float64
runtime                  float64
title                     object
dtype: object

## Check if there's any duplicates

In [748]:
# Show the sum of how many duplicates are
movies.duplicated().sum()

3

## Remove duplicates

In [749]:
movies = movies.drop_duplicates()

In [750]:
movies.shape

(4478, 10)

## Cleaning genres column


---

Genres column had empty arrays [] on the rows

In [751]:
# drop the values that have both [] in the genres column and 0 revenue
movies = movies.drop(movies[(movies['genres'] == '[]') & (movies['revenue'] == 0)].index)
# replace the [] in the genres column with 0 for better visibility (0 = no genres associated)
movies['genres'] = movies['genres'].replace('[]', 0)
# check if there's any value left with [] in the genres column
movies[movies['genres'] == '[]']


,belongs_to_collection,budget,genres,id,original_title,production_companies,release_date,revenue,runtime,title


## Cleaning production_companies

---

production_companies column had empty arrays [] on the rows

In [752]:
# replace the [] in the production_companies column with 0 for better visibility (0 = no production_companies associated)
movies['production_companies'] = movies['production_companies'].replace('[]', 0)
# check if there's any value left with [] in the prodyction_companies column
movies[movies['production_companies'] == '[]']

,belongs_to_collection,budget,genres,id,original_title,production_companies,release_date,revenue,runtime,title


## Checking if belongs_to_collection has any empty list either {} or []

In [753]:
# check if there's either [] or {} value in "belongs_to_collection" column
movies[movies['belongs_to_collection'] == '[]']
movies[movies['belongs_to_collection'] == '{}']

,belongs_to_collection,budget,genres,id,original_title,production_companies,release_date,revenue,runtime,title


## Check dataset after initial cleaning

In [754]:
movies.shape

(4414, 10)

## Convert data types for budget (object to float64), id (object to int) and release_date (object to *datetime*)

In [755]:
# convert budget from object to float64
movies['budget'] = movies['budget'].astype(float)
# convert id from object to int
movies['id'] = movies['id'].astype(int)
# convert release_date to datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies.dtypes

belongs_to_collection            object
budget                          float64
genres                           object
id                                int64
original_title                   object
production_companies             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
title                            object
dtype: object

## Rename the column id to movieId for better readability

In [756]:
movies = movies.rename(columns={"id":"movieId"})

## Check all the columns so far

In [757]:
movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'movieId',
       'original_title', 'production_companies', 'release_date', 'revenue',
       'runtime', 'title'],
      dtype='object')

NOTE: The columns 'genres', belongs_to_collection and production_companies appears to contain strings that look like dictionary representations, but they are stored as strings. For example, a typical string representation of a dictionary in Python looks like {'key': 'value'}.

So for instance in the belongs_to_collection:

String Representation:

Original Data: '{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}'
This data looks like a dictionary, but it's stored as a string.
Conversion using ast.literal_eval:

The ast.literal_eval function is a safe way to evaluate a string containing a Python literal or container display. It interprets the string as a literal and returns the corresponding Python object.
For example, ast.literal_eval("{'key': 'value'}") would return a dictionary object {'key': 'value'}.
Extracting 'name' Value:

After converting the string to a dictionary, the code then extracts the value associated with the 'name' key. For instance, {'id': 10194, 'name': 'Toy Story Collection'} would be transformed into the string 'Toy Story Collection'.
In summary, the process involves taking a string that looks like a dictionary, using ast.literal_eval to convert it into an actual dictionary, and then extracting specific values from that dictionary. This is a safe way to handle string representations of data structures in Python.

## Extract the values from "name" in each row from the column "genres"

In [758]:
import ast

# def clean_genres(x): Defines a function clean_genres that takes an input x.
def clean_genres(x):
    """ Checks if x is a string and not NaN.
        It uses isinstance to ensure that x is a string, and pd.notna checks if it's not NaN."""
    if isinstance(x, str) and pd.notna(x):
        """ This line is a list comprehension that processes the 'genres' column.
         It uses ast.literal_eval to safely evaluate the string representation of a list of dictionaries,
         replaces single quotes with double quotes to make it compatible with JSON format,
         and extracts the 'name' values after stripping leading and trailing whitespaces."""
        return [genre['name'].strip() for genre in ast.literal_eval(str(x).replace("'", "\""))]
    else:
        # Returns an empty list if x is not a valid string.
        return []

""" This line applies the clean_genres function to each element in the 'genres' column using the apply method,
creating a new column named 'cleaned_genres' in the DataFrame. """

movies['cleaned_genres'] = movies['genres'].apply(clean_genres)

# Assigns the values of 'cleaned_genres' to the 'genres' column, effectively replacing it.
movies['genres'] = movies['cleaned_genres']

# Removes the temporary 'cleaned_genres' column from the DataFrame.
movies.drop('cleaned_genres', axis=1, inplace=True)

# Display the updated DataFrame
movies['genres']

0           [Animation, Comedy, Family]
2                     [Romance, Comedy]
4                              [Comedy]
9         [Adventure, Action, Thriller]
12       [Family, Animation, Adventure]
                      ...              
45355                          [Comedy]
45358                          [Comedy]
45369                          [Comedy]
45371                       [Animation]
45382                 [Action, Foreign]
Name: genres, Length: 4414, dtype: object

## Extract each "name" value for each row in the column "belongs_to_collection" and separated only by commas

In [759]:
# This function takes an input x, evaluates the 'belongs_to_collection' string into a dictionary,
# and returns the 'name' value. If the input is NaN, it returns an empty string.
def extract_collection_name(x):
    if pd.notna(x):
        return ast.literal_eval(str(x))['name']
    else:
        return ''

# Apply the function to create a new column 'collection_name'
movies['collection_name'] = movies['belongs_to_collection'].apply(extract_collection_name)

# Replace 'belongs_to_collection' with 'collection_name'
movies['belongs_to_collection'] = movies['collection_name']

# Drop the temporary 'collection_name' column
movies.drop('collection_name', axis=1, inplace=True)

# Display the result
movies[['belongs_to_collection']]

,belongs_to_collection
0,Toy Story Collection
2,Grumpy Old Men Collection
4,Father of the Bride Collection
9,James Bond Collection
12,Balto Collection
...,...
45355,The Carry On Collection
45358,The Carry On Collection
45369,The Carry On Collection
45371,DC Super Hero Girls Collection


## Extracting only the names in each row from the column 'production_companies'


---
1.1 - show comparison

1.2 - replace old column for the new one with only the names extracted


In [760]:
""" Handling Mixed types of data with a function inclunding a try except:
The try block attempts to execute the list comprehension to extract 'name' values
from the evaluated string representation of a list of dictionaries (ast.literal_eval(str(x))).
The except (ValueError, TypeError) block catches errors of type ValueError or TypeError that may occur during the evaluation.
These errors could happen if the content of 'production_companies' is not a valid string representation of a list of dictionaries.
"""
def extract_production_companies_names(x):
    if pd.notna(x):
        try:
            return [company['name'] for company in ast.literal_eval(str(x))]
        except (ValueError, TypeError):
            return []
    else:
        """
        If an error is encountered during the evaluation, the function returns an empty list.
        This ensures that if the content of 'production_companies' is not in the expected format,
         the result will be an empty list instead of causing an error.
        """
        return []
# Apply the function to create a new column 'production_company_names'
movies['production_companies_names'] = movies['production_companies'].apply(extract_production_companies_names)

# Display the result comparison
movies[['production_companies', 'production_companies_names']]


,production_companies,production_companies_names
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]",[Pixar Animation Studios]
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[Warner Bros., Lancaster Gate]"
4,"[{'name': 'Sandollar Productions', 'id': 5842}...","[Sandollar Productions, Touchstone Pictures]"
9,"[{'name': 'United Artists', 'id': 60}, {'name'...","[United Artists, Eon Productions]"
12,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[Universal Pictures, Amblin Entertainment, Amb..."
...,...,...
45355,"[{'name': 'The Rank Organisation', 'id': 364},...","[The Rank Organisation, Adder]"
45358,"[{'name': 'The Rank Organisation', 'id': 364}]",[The Rank Organisation]
45369,"[{'name': 'The Rank Organisation', 'id': 364}]",[The Rank Organisation]
45371,"[{'name': 'Warner Bros. Animation', 'id': 2785}]",[Warner Bros. Animation]


In [761]:
# Assign values from the new column to the original 'production_companies' column
movies['production_companies'] = movies['production_companies_names']
# Drop the old  column which is now called production_companies_names
movies.drop('production_companies', axis=1, inplace=True)
movies

,belongs_to_collection,budget,genres,movieId,original_title,release_date,revenue,runtime,title,production_companies_names
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,Toy Story,1995-10-30,373554033.0,81.0,Toy Story,[Pixar Animation Studios]
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,Grumpier Old Men,1995-12-22,0.0,101.0,Grumpier Old Men,"[Warner Bros., Lancaster Gate]"
4,Father of the Bride Collection,0.0,[Comedy],11862,Father of the Bride Part II,1995-02-10,76578911.0,106.0,Father of the Bride Part II,"[Sandollar Productions, Touchstone Pictures]"
9,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",710,GoldenEye,1995-11-16,352194034.0,130.0,GoldenEye,"[United Artists, Eon Productions]"
12,Balto Collection,0.0,"[Family, Animation, Adventure]",21032,Balto,1995-12-22,11348324.0,78.0,Balto,"[Universal Pictures, Amblin Entertainment, Amb..."
...,...,...,...,...,...,...,...,...,...,...
45355,The Carry On Collection,0.0,[Comedy],24568,Follow That Camel,1967-09-01,0.0,95.0,Carry On Follow That Camel,"[The Rank Organisation, Adder]"
45358,The Carry On Collection,0.0,[Comedy],19307,Carry On Camping,1969-05-29,0.0,88.0,Carry On Camping,[The Rank Organisation]
45369,The Carry On Collection,0.0,[Comedy],21251,Carry On England,1976-10-01,0.0,89.0,Carry On England,[The Rank Organisation]
45371,DC Super Hero Girls Collection,0.0,[Animation],460135,LEGO DC Super Hero Girls: Brain Drain,2017-08-30,0.0,0.0,LEGO DC Super Hero Girls: Brain Drain,[Warner Bros. Animation]


## quickly review how the dataframe structure is so far and set movieId as the index

In [762]:
# assign movieId as the index of the dataset and sort the index in ascending order
movies = movies.set_index('movieId').sort_index()

movies

,belongs_to_collection,budget,genres,original_title,release_date,revenue,runtime,title,production_companies_names
movieId,,,,,,,,,
11,Star Wars Collection,11000000.0,"[Adventure, Action, Science Fiction]",Star Wars,1977-05-25,775398007.0,121.0,Star Wars,"[Lucasfilm, Twentieth Century Fox Film Corpora..."
12,Finding Nemo Collection,94000000.0,"[Animation, Family]",Finding Nemo,2003-05-30,940335536.0,100.0,Finding Nemo,[Pixar Animation Studios]
16,Heart of Gold Collection,12800000.0,"[Drama, Crime, Music]",Dancer in the Dark,2000-05-17,40031879.0,140.0,Dancer in the Dark,"[Fine Line Features, Zentropa Entertainments, ..."
21,The Endless Summer Collection,0.0,[Documentary],The Endless Summer,1966-06-15,0.0,95.0,The Endless Summer,[Bruce Brown Films]
22,Pirates of the Caribbean Collection,140000000.0,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: The Curse of the Bla...,2003-07-09,655011224.0,143.0,Pirates of the Caribbean: The Curse of the Bla...,"[Walt Disney Pictures, Jerry Bruckheimer Films]"
...,...,...,...,...,...,...,...,...,...
447113,Monster High Collection,0.0,"[Family, Animation]",Monster High: Electrified,2017-04-04,0.0,72.0,Monster High: Electrified,[Mattel Creations]
451644,DragonHeart Collection,0.0,[Adventure],Dragonheart: Battle for the Heartfire,2017-06-13,0.0,98.0,Dragonheart: Battle for the Heartfire,[]
452142,Maigret (Rowan Atkinson) Collection,0.0,"[TV Movie, Crime, Mystery, Thriller]",Maigrets Night at the Crossroads,2017-04-16,0.0,88.0,Maigrets Night at the Crossroads,[]


## TO DO: VERIFICAR SE O TITULO E TITULO ORIGINAL TÊM CARACTERES ESPECIAIS

In [763]:
# create a dataframe from the genres column
# genres = pd.DataFrame(movies.genres.str.split(',').tolist()).stack()
# movies['genres'] = movies['genres'].str.split(':').str[1]
# print(movies['genres'])

## FASE 1.2 TRANSFORMAÇÃO ***

## FASE 2 - Estatística descritiva

In [764]:
movies.describe()

,budget,revenue,runtime
count,4.414000e+03,4.414000e+03,4414.000000
mean,1.373724e+07,5.552799e+07,95.527866
std,3.661088e+07,1.654601e+08,28.849933
min,0.000000e+00,0.000000e+00,0.000000
25%,0.000000e+00,0.000000e+00,86.000000
50%,0.000000e+00,0.000000e+00,94.000000
75%,5.500000e+06,1.712609e+07,106.000000
max,3.800000e+08,2.787965e+09,476.000000
